In [2]:
import cv2
import numpy as np


In [ ]:
src = cv2.imread("coin.jpg")
src = cv2.resize(src, None, fx=0.2, fy=0.2)
cv2.imshow("src", src)

# 影線轉成灰階
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# # 二值化處理影像
ret, dst_binary = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# 使用自適應閾值進行二值化
# dst_binary = cv2.adaptiveThreshold(src_gray, 255,
#                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY_INV, 11, -2)
# 找尋影像內的輪廓
contours, hierarchy = cv2.findContours(dst_binary,
                                       cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)
# 繪製圖形輪廓
dst = cv2.drawContours(src, contours, -1, (0, 255, 0), 5)
# 初始化統計數據
triangle_count = 0                  # 三角形計數器
rectangle_count = 0                 # 矩形計數器
circle_count = 0                    # 圓形計數器

# 輪廓處理
for i, contour in enumerate(contours):
    # 輪廓近似
    epsilon = 0.02 * cv2.arcLength(contour, True)   # 根據輪廓周長調整
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 輪廓點數量
    num_points = len(approx)
    # 判斷類型
    if num_points == 3:                             # 三角形
        triangle_count += 1
        print(f"輪廓 {i} : 三角形, 輪廓點數量 = {num_points}")
    elif num_points == 4:                           # 矩形
        print(f"輪廓 {i} : 矩形,   輪廓點數量 = {num_points}")
        rectangle_count += 1
    elif num_points >= 8:                           # 圓形(近似為園)
        circle_count += 1
        print(f"輪廓 {i} : 圓形,   輪廓點數量 = {num_points}")
    else:
        print(f"輪廓 {i} : 其他形狀, 輪廓點數量 = {num_points}")

# 統計結果
print(f"三角形的輪廓總數量 : {triangle_count}")
print(f"矩形的輪廓總數量   : {rectangle_count}")
print(f"圓形的輪廓總數量   : {circle_count}")


# # 繪製文字
# font = cv2.FONT_HERSHEY_SIMPLEX
# for i in range(len(contours)):
#     cv2.putText(dst, str(i), contours[i][0][0], font, 1, (0, 0, 255), 2)
# for c in contours:                                   # 繪製中心點迴圈
#     M = cv2.moments(c)                               # 影像矩
#     Cx = int(M["m10"] / M["m00"])                    # 質心 x 座標
#     Cy = int(M["m01"] / M["m00"])                    # 質心 y 座標
#     cv2.circle(dst, (Cx, Cy), 5, (255, 0, 0), -1)    # 繪製中心點
cv2.imshow("result", dst)                            # 顯示結果影像
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
src = cv2.imread("coin.jpg")
src = cv2.resize(src, None, fx=0.2, fy=0.2)
cv2.imshow("src", src)

# 影線轉成灰階
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# 1. 影像預處理：使用 CLAHE 增強對比度
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_img = clahe.apply(src_gray)
# 使用 Otsu's method 進行二值化處理
ret, dst_binary = cv2.threshold(clahe_img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# 自適應閾值處理
# dst_binary = cv2.adaptiveThreshold(src_gray, 255,
#                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY_INV, 11, 2)
cv2.imshow('Binary Image', dst_binary)
# 尋找影像內的輪廓
contours, hierarchy = cv2.findContours(dst_binary,
                                       cv2.RETR_LIST,
                                       cv2.CHAIN_APPROX_SIMPLE)


# 創建一個空白圖片，用於繪製橢圓
result_img = np.copy(src)

# 遍歷所有輪廓
for contour in contours:
    # 只要點數足夠，就嘗試擬合
    if len(contour) > 5:
        ellipse = cv2.fitEllipse(contour)
        
        # 計算面積
        (center, axes, orientation) = ellipse
        ellipse_area = np.pi * axes[0] * axes[1]

        # 加上判斷式，確保分母不為零
        if ellipse_area > 0:
            # 只檢查輪廓面積是否大於一個很小的值，例如 1
            if cv2.contourArea(contour) > 140:
                # 繪製橢圓，這樣你可以看到所有被擬合出來的圖形
                cv2.ellipse(result_img, ellipse, (0, 255, 0), 5)

# 顯示結果，觀察所有被擬合出來的「橢圓」
cv2.imshow('All Fitted Ellipses', result_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

def process_coins(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 使用 cv2.resize() 將影像縮小
    # fx 和 fy 分別代表水平和垂直方向的縮放比例
    scale_factor = 1
    scaled_src = cv2.resize(src, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    output = scaled_src.copy()

    # 預處理：灰階和高斯模糊
    gray = cv2.cvtColor(scaled_src, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # 由於影像縮小了，圓形半徑的偵測範圍也需要調整
    # 這裡的 minRadius 和 maxRadius 也要根據縮小後的影像來調整
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=int(20 * scale_factor),  # 調整 minRadius
        maxRadius=int(70 * scale_factor)   # 調整 maxRadius
    )

    total_amount = 0

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 由於影像縮小了，硬幣的模板半徑也需要按比例調整
        coin_templates = {
            1: (int(30 * scale_factor), int(38 * scale_factor)),  # 1元硬幣的半徑範圍
            5: (int(45 * scale_factor), int(55 * scale_factor)),  # 5元硬幣的半徑範圍
            10: (int(60 * scale_factor), int(70 * scale_factor)) # 10元硬幣的半徑範圍
        }
        recognizer = CoinRecognizer(coin_templates)

        for i in circles[0, :]:
            center_x, center_y, radius = i[0], i[1], i[2]
            value = recognizer.recognize_coin(radius)

            if value > 0:
                total_amount += value
                cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                cv2.putText(output, str(value), (center_x - 10, center_y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 在圖片上顯示總金額
    cv2.putText(output, f"Total: {total_amount}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # 顯示結果
    cv2.imshow("Result", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 你的 CoinRecognizer 類別
class CoinRecognizer:
    def __init__(self, templates):
        self.templates = templates

    def recognize_coin(self, radius):
        for value, (min_r, max_r) in self.templates.items():
            if min_r <= radius <= max_r:
                return value
        return 0

# 調用函式並傳入你的圖片路徑
process_coins("coins2.jpg")